In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import os


print(os.getcwd())  # 현재 노트북 기준 경로 확인
df_gu = pd.read_csv("../../data/서울시 광진구 안심이 CCTV 연계현황.csv", encoding='cp949')
df_gu

/home/kang/code/gongmo/src/notebooks/stundrg


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,광진구,공원-광장-045-01,37.5460,127.1039,1,2022-12-01
1,광진구,공원-광장-045-02,37.5460,127.1039,1,2022-12-01
2,광진구,공원-광장-045-03,37.5460,127.1039,1,2022-12-01
3,광진구,공원-광장-045-비상벨,37.5460,127.1039,1,2022-12-01
4,광진구,공원-광장-046-01,37.5482,127.1006,1,2022-12-01
...,...,...,...,...,...,...
4086,광진구,주차-화양-034-03,37.5421,127.0714,1,2022-12-01
4087,광진구,주차-화양-035-01,37.5409,127.0709,1,2022-12-01
4088,광진구,주차-화양-035-02,37.5409,127.0709,1,2022-12-01
4089,광진구,주차-화양-035-03,37.5409,127.0709,1,2022-12-01
